### Convolution Neural Net

#### We should have enough to put together a cnn now.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# get device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# predefine some hyper parameter
num_epochs = 5
batch_size = 4
learning_rate = 0.001

# dataset has PILImage images of range[0, 1], we transform them to tensors of formalized range of [-1, 1]
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
])

Not sure why we are normalizing to mean and std of 0.5, instead of 0; need to investigate.

In [2]:
?transforms.Normalize

Now we get some data.

In [3]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=transforms)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

classess = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [4]:
# Implement the Conv net, and get the key components like the loss function and optimizer.

class ConvNet(nn.Module):
    def __init__(self):
        pass
    
    def forward(self, x):
        pass
    
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()  # the usual, for multi-label classification
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

ModuleAttributeError: 'ConvNet' object has no attribute '_modules'

Hard coded training loop:

In [ ]:
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # original shape: [4, 3, 32, 32] = 4, 3,  1024 
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward and optimize weight
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # logging
        if (i+1) % 1000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
            
print(f'Finished training')

Simple Analysis:

In [5]:
# remember to torch.no_grad() as we don't want these calculation to go into the computation graph

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        # max returns (value, index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc}%')
    
    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc}%')

TypeError: 'module' object is not callable